In [ ]:
import sys; sys.path.append('..')
import elastic_rods, sparse_matrices, pickle, scipy, linkage_vis, numpy as np, time
from numpy.random import uniform as random
from linkage_vis import LinkageViewer as Viewer
import matplotlib.pyplot as plt

In [ ]:
npts = 10
r = elastic_rods.ElasticRod(np.linspace([0, 0, 0], [5, 0, 0], npts))
r.setMaterial(elastic_rods.RodMaterial('ellipse', 2000, 0.3, [0.1, 0.05]))
view = Viewer(r)
view.show()

This codebase implements two variants of the bending energy:
the version presented in the discrete viscous threads paper (Bergou2010) which
non-physically averages the two incident edges' material frames to compute the
material curvatures at a vertex, and a more physically meaningful version in the
spirit of the original discrete elastic rods paper (Bergou2008) that averages
the bending energies resulting from the two distinct material frames.
The bending energy implementation can be chosen as follows:

In [ ]:
# r.bendingEnergyType = elastic_rods.BendingEnergyType.Bergou2010
r.bendingEnergyType = elastic_rods.BendingEnergyType.Bergou2008

In [ ]:
# Perturb the rod into a general configuration
r.setDeformedConfiguration([[5 * t, np.cos(2 * np.pi * t), 0] for t in np.linspace(0, 1, npts)], np.pi / 4 * random(-1, 1, r.numEdges()))
r.setDoFs(r.getDoFs() + 1e-1 * random(-1, 1, size=r.numDoF()))
view.update()

Run the finite difference validation: we pick a random direction in the rod's configuration space and test whether the finite difference approximation to the directional derivative converges to the directional derivative computed using the analytical gradient. We note a perfect second order convergence in these plots (until numerical cancellation kicks in), indicating that the analytical gradients and Hessians are accurate.

In [ ]:
import finite_diff
finite_diff.gradient_convergence_plot(r)

In [ ]:
r.updateSourceFrame() # Hessian formulas are only accurate with an up-to-date source frame
finite_diff.hessian_convergence_plot(r)

In [ ]:
# Display the discrepancy of finite differences of the "infinitesimal transport" (post-update) gradient
r.updateSourceFrame() # Hessian formulas are only accurate with an up-to-date source frame
finite_diff.hessian_convergence_plot(r, infinitesimalTransportGradient=True)